In [1]:
{-# LANGUAGE InstanceSigs #-}
newtype Reader e a = Reader { runReader :: e -> a }

In [2]:
instance Functor (Reader e) where
    fmap :: (a -> b) -> Reader e a -> Reader e b
    fmap f reader = Reader $ f . runReader reader

In [3]:
instance Applicative (Reader e) where
    pure :: a -> Reader e a
    pure a = Reader $ const a
    (<*>) :: Reader e (a -> b) -> Reader e a -> Reader e b
    f <*> a = Reader $ \e -> let
        f' = runReader f e
        a' = runReader a e
        in f' a'

In [4]:
instance Monad (Reader e) where
    (>>=) :: Reader e a -> (a -> Reader e b) -> Reader e b
    a >>= f = Reader $ \e -> let
        a' = runReader a e
        b  = f a'
        in runReader b e

In [5]:
newtype ReaderT e m a = ReaderT { runReaderT :: e -> m a }

In [6]:
instance Functor m => Functor (ReaderT e m) where
    fmap :: (a -> b) -> ReaderT e m a -> ReaderT e m b
    fmap f (ReaderT readert) = ReaderT $ \e -> let
        ma = readert e
        in f <$> ma

In [7]:
instance Monad m => Applicative (ReaderT e m) where
    pure :: a -> ReaderT e m a
    pure a = ReaderT $ const (pure a)
    
    (<*>) :: ReaderT e m (a -> b) -> ReaderT e m a -> ReaderT e m b
    (<*>) (ReaderT mf) (ReaderT ma) = ReaderT $ \e -> do
        f <- mf e
        a <- ma e
        pure (f a)

In [8]:
instance Monad m => Monad (ReaderT e m) where
    (>>=) :: ReaderT e m a -> (a -> ReaderT e m b) -> ReaderT e m b
    (>>=) (ReaderT ma) f = ReaderT $ \e -> do
        a <- ma e
        let (ReaderT mb) = f a
        mb e

In [9]:
import Control.Monad.Trans.Class

instance MonadTrans (ReaderT e) where
    lift :: m a -> ReaderT e m a
    lift ma = ReaderT $ const ma

In [10]:
ask :: Monad m => ReaderT e m e
ask = ReaderT $ \e -> pure e

In [11]:
guessing :: ReaderT String IO ()
guessing = do
    lift $ putStrLn "Make a guess!"
    g <- lift $ getLine
    correct <- ask
    if g == correct
        then do
            lift $ putStrLn "You got it right!"
        else do
            lift $ putStrLn "Nope!"
            guessing